# Projeto 1 - Ciência dos Dados

Nome: Bruno Boldrim Saboya

Nome: Murilo Prado Weyne

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
import string
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bbsab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'XBOX.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo XBOX.xlsx, tudo certo para prosseguir com a prova!


In [3]:
def cleanup(text):
    limpar = ['@', '(', ')', '[', ']', '+', '-', '\n', '\t', 'rt']
    punctuation = '[!-.:?;\/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    for t in limpar:
        text_subbed = text_subbed.replace(t, ' ')
    text_subbed=text_subbed.replace('ã', 'a')
    text_subbed=text_subbed.replace('â', 'a')
    text_subbed=text_subbed.replace('á', 'a')
    text_subbed=text_subbed.replace('à', 'a')
    text_subbed=text_subbed.replace('é', 'e')
    text_subbed=text_subbed.replace('è', 'e')
    text_subbed=text_subbed.replace('ê', 'e')
    text_subbed=text_subbed.replace('î', 'i')
    text_subbed=text_subbed.replace('í', 'i')
    text_subbed=text_subbed.replace('ì', 'i')
    text_subbed=text_subbed.replace('ó', 'o')
    text_subbed=text_subbed.replace('ò', 'o')
    text_subbed=text_subbed.replace('õ', 'o')
    text_subbed=text_subbed.replace('ô', 'o')
    text_subbed=text_subbed.replace('ú', 'u')
    text_subbed=text_subbed.replace('ù', 'a')
    text_subbed=text_subbed.replace('tbm', 'tambem')   
    text_subbed=text_subbed.replace('tb', 'tambem')
    text_subbed=text_subbed.replace('vc', 'voce')
    text_subbed=text_subbed.replace('nois', 'nos')
    return text_subbed

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
train = pd.read_excel(filename)
train.head(20)

,Treinamento,Relevancia
0,@mr_garcez tava em dúvida do xbox one é\ne o ps3,1
1,@sonystasensato @italotravassoss @fuck_off_mat...,1
2,rt @capitaocritica: então galera hoje quê é o ...,1
3,as pré-vendas do xbox series s e xbox series x...,1
4,rt @sbtonline: mais 5 jogos vão entrar para o ...,1
5,@capitaorumor vou de xbox pois no ps5 além de ...,1
6,rt @chief117_com_br: não queria desviar o foco...,0
7,rt @siathosa: hyper x cloud stinger core verme...,0
8,rt @pacta_serv: hj será a última chance da son...,1
9,rt @xboxbr: a arma mais poderosa da época medi...,0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste
0,@drakesincero1 se sair esta parceira o xbox va...
1,@verligameplay @guisouzadias comecei nesse aqu...
2,rt @kah_xp: 900 jogos concluídos 100% no xbox!...
3,hoje é dia de xbox mil grau chorar no twitter ...
4,@rebekah_soaress me add no xbox? ou omlet pra ...


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Consideramos relevantes os tweets que falam sobre o produto e suas características e não relevantes aqueles que não falam do produto e/ou citam o nome sem falar das características.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
train.loc[:,'Relevancia'] = train['Relevancia'].astype('category')
train['Relevancia'].cat.categories = ["Irrelevante", "Relevante"]
train

,Treinamento,Relevancia
0,@mr_garcez tava em dúvida do xbox one é\ne o ps3,Relevante
1,@sonystasensato @italotravassoss @fuck_off_mat...,Relevante
2,rt @capitaocritica: então galera hoje quê é o ...,Relevante
3,as pré-vendas do xbox series s e xbox series x...,Relevante
4,rt @sbtonline: mais 5 jogos vão entrar para o ...,Relevante
...,...,...
295,@renansimeonoff1 @xgrey_wolf93 a maior propaga...,Irrelevante
296,"@murilo_beeble rapaz, ver youtube ficou bom de...",Irrelevante
297,rt @bru_leonel1986: @renansimeonoff1 @xgrey_wo...,Irrelevante
298,"@luizknight agora sonysta migra para o xbox, j...",Irrelevante


In [7]:
relevante1 = train['Relevancia'] == 'Relevante'
relevante = train.loc[relevante1,['Treinamento']].copy()
relevante

,Treinamento
0,@mr_garcez tava em dúvida do xbox one é\ne o ps3
1,@sonystasensato @italotravassoss @fuck_off_mat...
2,rt @capitaocritica: então galera hoje quê é o ...
3,as pré-vendas do xbox series s e xbox series x...
4,rt @sbtonline: mais 5 jogos vão entrar para o ...
...,...
289,"@eduoiiveira eu não vi, a única coisa que eu v..."
291,rt @ppggnaarea: tga2019 phill surpreendeu o mu...
292,"rt @xboxbr: pela primeira vez no console, uma ..."
294,@opcxbox @xboxbr já é estratégico...\nquem não...


In [8]:
irrelevante1 = train['Relevancia'] == 'Irrelevante'
irrelevante = train.loc[irrelevante1,['Treinamento']].copy()
irrelevante

,Treinamento
6,rt @chief117_com_br: não queria desviar o foco...
7,rt @siathosa: hyper x cloud stinger core verme...
9,rt @xboxbr: a arma mais poderosa da época medi...
10,rt @chief117_com_br: e pelo jeito conseguiram ...
11,"@xkaioh @xboxbr ""ai quem falou merda foi ela""\..."
...,...
293,#osplayers o xbox não aguentou essa geração e ...
295,@renansimeonoff1 @xgrey_wolf93 a maior propaga...
296,"@murilo_beeble rapaz, ver youtube ficou bom de..."
297,rt @bru_leonel1986: @renansimeonoff1 @xgrey_wo...


In [9]:
'''realizamos aqui um split da lista dos tweets relevantes e irrelevantes com a função .tolist, cleanup, e .split
'''

lista_relevante = relevante['Treinamento'].tolist()
lista_irrelevante = irrelevante['Treinamento'].tolist()

lista_relevante_split = []
for i in lista_relevante:
    i = cleanup(i)
    i = i.split()
    lista_relevante_split.append(i)

lista_irrelevante_split = []
for k in lista_irrelevante:
    k = cleanup(k)
    k = k.split()
    lista_irrelevante_split.append(k)

In [10]:
texto_relevante = ''.join(relevante.Treinamento)
texto_relevanteClean = cleanup(texto_relevante)

texto_irrelevante = ''.join(irrelevante.Treinamento)
texto_irrelevanteClean = cleanup(texto_irrelevante)

In [11]:
'''Implementamos um stopwors para remover as palavras irrelevantes e que nao interfeririram na conta
da probalilidade
'''

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

text = texto_relevanteClean
text_tokens = word_tokenize(text)

texto_relevante_clean_sw = [word for word in text_tokens if not word in stopwords.words()]

In [12]:
text = texto_irrelevanteClean
text_tokens = word_tokenize(text)

texto_irrelevante_clean_sw = [word for word in text_tokens if not word in stopwords.words()]

In [13]:
''' Realizamos o cálculo da frequêcia absoluta dos relevantes e irrelevantes
'''


relevante_series = pd.Series(texto_relevante_clean_sw)
irrelevante_series = pd.Series(texto_irrelevante_clean_sw)

FreqAbsRel = relevante_series.value_counts()
FreqRelRel = relevante_series.value_counts(True)

FreqAbsIrr = irrelevante_series.value_counts()
FreqRelIrr = irrelevante_series.value_counts(True)

texto_total = texto_relevanteClean + texto_irrelevanteClean
lista_texto_total = texto_total.split()
total_series = pd.Series(lista_texto_total)

FreqAbsTot = total_series.value_counts()
FreqRelTot = total_series.value_counts(True)

total_relevante = len(FreqAbsRel)
total_total = len(FreqAbsTot)
total_irrelevante = len(FreqAbsIrr)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [30]:
lista_prob_R = []
lista_prob_Ir = []

bayes = []

for frase in lista_relevante_split:
    probR = 1
    probIrr = 1
    for palavra in frase:
        if palavra in FreqAbsRel:
            probR *= ((FreqAbsRel[palavra] + 1)/(total_relevante + total_total))
        if palavra not in FreqAbsRel:
            probR *= ((1)/(total_total+total_relevante))
        if palavra in FreqAbsIrr:
            probIrr *= ((FreqAbsIrr[palavra] + 1)/(total_irrelevante + total_total))
        if palavra not in FreqAbsIrr:
            probIrr *= ((1)/(total_irrelevante + total_total))
    if (probR) > (probIrr):
        train['NaiveBayes'] = 'Relevante'
    else:
        train['NaiveBayes'] = 'Irrelevante'
    lista_prob_R.append(probR)
    lista_prob_Ir.append(probIrr)


___
### Concluindo

Nossa conclusão com esse projetp é que esse classificador serve para identificar os tweets relevantes para a empresa para ela então identificar onde pode melhorar e direcionar investimentos para determinados setores. Entretanto o nosso classificador ainda precisa de melhorias de aperfeiçoamento.
O motivo do qual a empresa deva continuar bancando esse trabalho é que o potencial de ganho da empresa é bem grande e o nosso código ainda precisa de melhorias, como realizar uma limpeza nos tweets.
As mensagen de dupla negação são interpretadas como apenas uma negação, portanto, é classificada como irrelevante. Seria necessário um aperfeiçoamento no código para que esses tweets sejam classificados como relevantes.


Naive Bayes além de poder ser utilizado como este classificador, também pode ser usado em diversar outras situações. Dentre elas:
- Decodificação de mensagens, comparando as probabilidades de uma palavra pertencer a uma linguagem ou a outra.

- Prever resultados de jogos de disputa, analisando as probabilidades condicionais de um jogo que levam a uma eventual vitória ou derrota.

Entre outras possíveis utilidades.

Algumas melhorias que podem ser feitas são:

- Remover as stopwords (palavras comuns que atrapalham na qualidade da classificação)

- Agrupar palavras. Exemplo: plurais (jogo, jogos, jogões)

Para remover as stopwords é necessária a instalação da biblioteca [nltk].

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**